# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

120000

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AEsharp_diff'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAEsharp_diff\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 32)      128       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 32)      1056      
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                40960032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [26]:
#from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': sharp_diff})

Model: "pokemonAEsharp_diff"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                40961344  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       3962124   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 44,923,468
Trainable params: 44,923,398
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAEsharp_diff\\csv_dir\\pokemonAEsharp_diff.csv'

In [33]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=750,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=250
)

Train for 10 steps, validate for 10 steps


Epoch 1/10
10/10 [==============================] - ETA: 35s - loss: 7.3205 - psnr: 7.3411 - ssmi: 0.0351 - sharp_diff: 7.276 - ETA: 17s - loss: 6.8875 - psnr: 7.2692 - ssmi: 0.0297 - sharp_diff: 6.833 - ETA: 10s - loss: 6.7993 - psnr: 7.2331 - ssmi: 0.0290 - sharp_diff: 6.741 - ETA: 7s - loss: 6.7130 - psnr: 7.2011 - ssmi: 0.0279 - sharp_diff: 6.652 - ETA: 5s - loss: 6.6858 - psnr: 7.1867 - ssmi: 0.0276 - sharp_diff: 6.62 - ETA: 3s - loss: 6.6259 - psnr: 7.1814 - ssmi: 0.0276 - sharp_diff: 6.56 - ETA: 2s - loss: 6.5461 - psnr: 7.1690 - ssmi: 0.0265 - sharp_diff: 6.48 - ETA: 1s - loss: 6.4840 - psnr: 7.1753 - ssmi: 0.0259 - sharp_diff: 6.41 - ETA: 0s - loss: 6.4526 - psnr: 7.1788 - ssmi: 0.0257 - sharp_diff: 6.38 - 9s 924ms/step - loss: 6.3797 - psnr: 7.1737 - ssmi: 0.0250 - sharp_diff: 6.3094 - val_loss: 11.9086 - val_psnr: 7.5272 - val_ssmi: 0.1423 - val_sharp_diff: 11.9063


Epoch 2/10
10/10 [==============================] - ETA: 2s - loss: 5.3762 - psnr: 7.0521 - ssmi: 0.0146 - sharp_diff: 5.27 - ETA: 2s - loss: 5.4506 - psnr: 7.0100 - ssmi: 0.0160 - sharp_diff: 5.35 - ETA: 1s - loss: 5.5093 - psnr: 6.9898 - ssmi: 0.0161 - sharp_diff: 5.41 - ETA: 1s - loss: 5.4743 - psnr: 7.0025 - ssmi: 0.0162 - sharp_diff: 5.37 - ETA: 1s - loss: 5.4403 - psnr: 6.9742 - ssmi: 0.0160 - sharp_diff: 5.34 - ETA: 1s - loss: 5.3858 - psnr: 6.9765 - ssmi: 0.0160 - sharp_diff: 5.28 - ETA: 0s - loss: 5.3295 - psnr: 6.9653 - ssmi: 0.0160 - sharp_diff: 5.22 - ETA: 0s - loss: 5.3175 - psnr: 6.9761 - ssmi: 0.0165 - sharp_diff: 5.21 - ETA: 0s - loss: 5.3077 - psnr: 6.9839 - ssmi: 0.0167 - sharp_diff: 5.20 - 4s 396ms/step - loss: 5.2485 - psnr: 6.9843 - ssmi: 0.0165 - sharp_diff: 5.1402 - val_loss: 11.0676 - val_psnr: 7.5036 - val_ssmi: 0.1197 - val_sharp_diff: 11.0611


Epoch 3/10
10/10 [==============================] - ETA: 2s - loss: 4.6014 - psnr: 6.8963 - ssmi: 0.0135 - sharp_diff: 4.47 - ETA: 2s - loss: 4.6263 - psnr: 6.9256 - ssmi: 0.0134 - sharp_diff: 4.49 - ETA: 1s - loss: 4.6262 - psnr: 6.8471 - ssmi: 0.0147 - sharp_diff: 4.49 - ETA: 1s - loss: 4.6010 - psnr: 6.8973 - ssmi: 0.0147 - sharp_diff: 4.47 - ETA: 1s - loss: 4.5235 - psnr: 6.8618 - ssmi: 0.0141 - sharp_diff: 4.38 - ETA: 1s - loss: 4.5070 - psnr: 6.8619 - ssmi: 0.0139 - sharp_diff: 4.37 - ETA: 0s - loss: 4.4547 - psnr: 6.8811 - ssmi: 0.0137 - sharp_diff: 4.31 - ETA: 0s - loss: 4.3881 - psnr: 6.8658 - ssmi: 0.0132 - sharp_diff: 4.24 - ETA: 0s - loss: 4.3342 - psnr: 6.8675 - ssmi: 0.0129 - sharp_diff: 4.19 - 4s 393ms/step - loss: 4.2868 - psnr: 6.8538 - ssmi: 0.0127 - sharp_diff: 4.1405 - val_loss: 9.9742 - val_psnr: 7.4423 - val_ssmi: 0.0885 - val_sharp_diff: 9.9619


Epoch 4/10
10/10 [==============================] - ETA: 2s - loss: 3.6818 - psnr: 6.6652 - ssmi: 0.0093 - sharp_diff: 3.51 - ETA: 2s - loss: 3.7577 - psnr: 6.6669 - ssmi: 0.0099 - sharp_diff: 3.58 - ETA: 1s - loss: 3.8012 - psnr: 6.6774 - ssmi: 0.0102 - sharp_diff: 3.62 - ETA: 1s - loss: 3.7602 - psnr: 6.6330 - ssmi: 0.0097 - sharp_diff: 3.58 - ETA: 1s - loss: 3.7186 - psnr: 6.6239 - ssmi: 0.0096 - sharp_diff: 3.54 - ETA: 1s - loss: 3.7112 - psnr: 6.6032 - ssmi: 0.0097 - sharp_diff: 3.53 - ETA: 0s - loss: 3.6991 - psnr: 6.6237 - ssmi: 0.0100 - sharp_diff: 3.52 - ETA: 0s - loss: 3.6408 - psnr: 6.6109 - ssmi: 0.0098 - sharp_diff: 3.45 - ETA: 0s - loss: 3.6131 - psnr: 6.6094 - ssmi: 0.0097 - sharp_diff: 3.43 - 4s 394ms/step - loss: 3.5799 - psnr: 6.6024 - ssmi: 0.0096 - sharp_diff: 3.3966 - val_loss: 9.6771 - val_psnr: 7.4950 - val_ssmi: 0.0898 - val_sharp_diff: 9.6636


...

Epoch 6/10
10/10 [==============================] - ETA: 2s - loss: 2.9110 - psnr: 6.6502 - ssmi: 0.0092 - sharp_diff: 2.69 - ETA: 2s - loss: 2.8719 - psnr: 6.5629 - ssmi: 0.0086 - sharp_diff: 2.65 - ETA: 1s - loss: 2.8677 - psnr: 6.5363 - ssmi: 0.0083 - sharp_diff: 2.64 - ETA: 1s - loss: 2.8605 - psnr: 6.5181 - ssmi: 0.0081 - sharp_diff: 2.64 - ETA: 1s - loss: 2.8461 - psnr: 6.5010 - ssmi: 0.0079 - sharp_diff: 2.62 - ETA: 1s - loss: 2.8108 - psnr: 6.4795 - ssmi: 0.0077 - sharp_diff: 2.58 - ETA: 0s - loss: 2.8117 - psnr: 6.4688 - ssmi: 0.0077 - sharp_diff: 2.59 - ETA: 0s - loss: 2.7905 - psnr: 6.4479 - ssmi: 0.0075 - sharp_diff: 2.56 - ETA: 0s - loss: 2.7890 - psnr: 6.4557 - ssmi: 0.0075 - sharp_diff: 2.56 - 5s 507ms/step - loss: 2.7750 - psnr: 6.4545 - ssmi: 0.0074 - sharp_diff: 2.5511 - val_loss: 8.4010 - val_psnr: 7.5202 - val_ssmi: 0.0684 - val_sharp_diff: 8.3782


Epoch 7/10
10/10 [==============================] - ETA: 2s - loss: 2.5420 - psnr: 6.3581 - ssmi: 0.0066 - sharp_diff: 2.30 - ETA: 2s - loss: 2.6407 - psnr: 6.4209 - ssmi: 0.0073 - sharp_diff: 2.40 - ETA: 1s - loss: 2.6231 - psnr: 6.3709 - ssmi: 0.0068 - sharp_diff: 2.39 - ETA: 1s - loss: 2.6343 - psnr: 6.3998 - ssmi: 0.0070 - sharp_diff: 2.40 - ETA: 1s - loss: 2.6449 - psnr: 6.4002 - ssmi: 0.0069 - sharp_diff: 2.41 - ETA: 1s - loss: 2.6316 - psnr: 6.4242 - ssmi: 0.0071 - sharp_diff: 2.39 - ETA: 0s - loss: 2.6201 - psnr: 6.4168 - ssmi: 0.0071 - sharp_diff: 2.38 - ETA: 0s - loss: 2.6075 - psnr: 6.4210 - ssmi: 0.0071 - sharp_diff: 2.37 - ETA: 0s - loss: 2.5979 - psnr: 6.4286 - ssmi: 0.0071 - sharp_diff: 2.36 - 4s 381ms/step - loss: 2.5781 - psnr: 6.4156 - ssmi: 0.0070 - sharp_diff: 2.3424 - val_loss: 8.8891 - val_psnr: 7.4904 - val_ssmi: 0.0737 - val_sharp_diff: 8.8710


Epoch 8/10
10/10 [==============================] - ETA: 2s - loss: 2.4870 - psnr: 6.0586 - ssmi: 0.0051 - sharp_diff: 2.25 - ETA: 2s - loss: 2.4682 - psnr: 6.2004 - ssmi: 0.0057 - sharp_diff: 2.23 - ETA: 1s - loss: 2.4349 - psnr: 6.2299 - ssmi: 0.0059 - sharp_diff: 2.19 - ETA: 1s - loss: 2.4256 - psnr: 6.2785 - ssmi: 0.0060 - sharp_diff: 2.18 - ETA: 1s - loss: 2.4326 - psnr: 6.2772 - ssmi: 0.0061 - sharp_diff: 2.19 - ETA: 1s - loss: 2.4163 - psnr: 6.3001 - ssmi: 0.0061 - sharp_diff: 2.17 - ETA: 0s - loss: 2.4268 - psnr: 6.2970 - ssmi: 0.0062 - sharp_diff: 2.18 - ETA: 0s - loss: 2.4244 - psnr: 6.3133 - ssmi: 0.0062 - sharp_diff: 2.18 - ETA: 0s - loss: 2.4255 - psnr: 6.3240 - ssmi: 0.0062 - sharp_diff: 2.18 - 4s 379ms/step - loss: 2.4128 - psnr: 6.3313 - ssmi: 0.0063 - sharp_diff: 2.1718 - val_loss: 8.6721 - val_psnr: 7.4893 - val_ssmi: 0.0700 - val_sharp_diff: 8.6522


Epoch 9/10
10/10 [==============================] - ETA: 2s - loss: 2.3661 - psnr: 6.2937 - ssmi: 0.0057 - sharp_diff: 2.12 - ETA: 2s - loss: 2.3457 - psnr: 6.3170 - ssmi: 0.0059 - sharp_diff: 2.10 - ETA: 1s - loss: 2.3252 - psnr: 6.2938 - ssmi: 0.0060 - sharp_diff: 2.08 - ETA: 1s - loss: 2.3434 - psnr: 6.2961 - ssmi: 0.0060 - sharp_diff: 2.09 - ETA: 1s - loss: 2.3401 - psnr: 6.3108 - ssmi: 0.0062 - sharp_diff: 2.09 - ETA: 1s - loss: 2.3283 - psnr: 6.3146 - ssmi: 0.0062 - sharp_diff: 2.08 - ETA: 0s - loss: 2.3205 - psnr: 6.3155 - ssmi: 0.0062 - sharp_diff: 2.07 - ETA: 0s - loss: 2.3380 - psnr: 6.3068 - ssmi: 0.0061 - sharp_diff: 2.09 - ETA: 0s - loss: 2.3367 - psnr: 6.3106 - ssmi: 0.0062 - sharp_diff: 2.09 - 4s 376ms/step - loss: 2.3376 - psnr: 6.3077 - ssmi: 0.0062 - sharp_diff: 2.0914 - val_loss: 8.5134 - val_psnr: 7.4229 - val_ssmi: 0.0669 - val_sharp_diff: 8.4921


Epoch 10/10
10/10 [==============================] - ETA: 2s - loss: 2.4398 - psnr: 6.3866 - ssmi: 0.0074 - sharp_diff: 2.19 - ETA: 2s - loss: 2.3930 - psnr: 6.3293 - ssmi: 0.0067 - sharp_diff: 2.14 - ETA: 1s - loss: 2.3302 - psnr: 6.3311 - ssmi: 0.0066 - sharp_diff: 2.08 - ETA: 1s - loss: 2.3297 - psnr: 6.4062 - ssmi: 0.0068 - sharp_diff: 2.08 - ETA: 1s - loss: 2.3798 - psnr: 6.5137 - ssmi: 0.0086 - sharp_diff: 2.13 - ETA: 1s - loss: 2.3533 - psnr: 6.4964 - ssmi: 0.0082 - sharp_diff: 2.10 - ETA: 0s - loss: 2.3474 - psnr: 6.4835 - ssmi: 0.0079 - sharp_diff: 2.09 - ETA: 0s - loss: 2.3346 - psnr: 6.4587 - ssmi: 0.0077 - sharp_diff: 2.08 - ETA: 0s - loss: 2.3104 - psnr: 6.4368 - ssmi: 0.0074 - sharp_diff: 2.05 - 4s 392ms/step - loss: 2.2961 - psnr: 6.4253 - ssmi: 0.0073 - sharp_diff: 2.0448 - val_loss: 7.8138 - val_psnr: 7.4350 - val_ssmi: 0.0555 - val_sharp_diff: 7.7863
